In [ ]:
import numpy as np
f = lambda x: np.sin(x)

## Regra do Trapézio

$$I(a,b) = \int_{a}^{b}{f(x)}dx$$
       
$$I(a,b) \approx h\left[\frac{1}{2}f(a) + \frac{1}{2}f(b) + \sum_{k=1}^{N-1}{f(a+kh)}\right]$$
$$\\$$
Erro:
$$\epsilon_{1} = \frac{1}{12}h^2[f'(a)-f'(b)]$$
$$\$$
$$\epsilon_{2} = \frac{1}{3} \left( I_{2} - I_{1} \right)$$

In [ ]:
def solveTrapezio(func, a, b, N):
    x = np.linspace(a, b, N + 1)
    f = func(x)
    dx = x[1]-x[0]
    
    soma = 0.5 * (f[0] + f[-1])
    return (f[1:-1].sum() + soma) * dx


def solveTrapezioErro(func, a, b, ERRO):
    
    def Ii2(func, a, b, N):
        x = np.linspace(a, b, N + 1)
        f = func(x)
        dx = x[1]-x[0]
        soma = 0
        return (f[1:-1:2].sum()) * dx

    R = np.empty((100,100))
    R[0,0] = (b-a)*(func(b)+func(a))/2
    
    N = 1
    i = 0
    epsilon = 1

    while epsilon > ERRO:
        N *= 2
        i+=1
        R[i,0] = R[i-1,0]/2 + Ii2(func, 0, 1, N)
    
        tabela = "\n"
        tabela += "N = {} \n".format(N)
        tabela +="I{} = {}\n".format(i,R[i,0])
    
        epsilon = abs(R[i,0]-R[i-1,0])/3
        tabela += f"Erro: {epsilon}"
        
        print(tabela)


In [ ]:
solveTrapezioErro(f,0,1,1e-6)

## Regra de Simpson

$$I(a,b) \approx \frac{1}{3}h\left[f(a)+f(b)+4\sum_{k\space odd}^{N-1}{f(a+kh)}+2\sum_{k\space even}^{N-2}{f(a+kh)}\right]$$
$$\\$$
Erro:
$$\epsilon_{1} = \frac{1}{90}h^4[f'''(a)-f'''(b)]$$
$$\$$
$$\epsilon_{2} = \frac{1}{15} \left( I_{2} - I_{1} \right)$$

In [ ]:
def solveSimpson(func, a, b, N): 
    x = np.linspace(a, b, 2*N+1)
    f = func(x)
    h = x[1]-x[0]
    return (h/3) * (f[0] + f[-1] + 2*sum(f[2:-1:2]) + 4*sum(f[1:-1:2]))


def solveSimpsonErro(func, a, b, ERRO):

    def Ii(f, a, b, N):
        h = (b - a) / N 
        s = 0
        for k in range(1,N//2+1):
            s += f(a + (2*k-1)*h)    
        return s*h

    R = np.empty((100,100))
    R[0,0] = (b-a)*(f(b)+f(a))/2
    
    epsilon = 1
    N = 1
    i = 0

    while epsilon> ERRO:
    
        N *= 2
        i+=1
    
        R[i,0] = R[i-1,0]/2 + Ii(f, 0, 1, N)
    
        tabela = "\n"
        tabela +="I{} = {:.8f}\n".format(i, R[i,0])
    
        for m in range(i):
            R[i,m+1] = R[i,m] + (R[i,m] - R[i-1,m])/(4**(m+1)-1)
        
            tabela += f"R[{i},{m+1}] = {R[i,m]:.8f}  |  "
    
        epsilon = abs(R[i,m+1]-R[i-1,m])
        tabela += f"\nErro: {epsilon:.8f}"

        print(tabela)


In [ ]:
solveSimpsonErro(f, 0, 1, 1e-6)

## Método de Romberg
$$S_{i}\ = \frac{1}{3}\left[f(a) + f(b) + 2\sum_{k par}^{N-1}{f(a+khi)}\right]$$ 
$$\\$$
$$T_{i}\ = \frac{2}{3}\sum_{k impar}^{N}{f(a+khi)}$$ 

$$S_{i}\ = S_{i-1} + T_{i-1}$$

$$I_{i}\ = h_{i}(S_{i} + 2T_{i})$$
Erro:
$$\epsilon_{i} = \frac{1}{5}(I_{i}-I_{i-1})$$

In [ ]:
def solveRomberg(func, a, b, ERRO):

    N = 2
    h = (b - a) / N
    keven = np.arange(2,N-1,2)
    kodd = np.arange(1,N,2)
    
    S = (1/3) * (func(a) + func(b) + np.sum(func(a + keven*h)))
    T = (2/3) * (np.sum(func(a + kodd*h)))
    I1 = h * (S + 2*T)
    
    epsilon = 1
    i = 0

    while epsilon> ERRO:
        
        i += 1
        N = N*2
        h = (b-a)/N
        S = S+T
        kodd = np.arange(1,N,2)
        T = (2/3) * (np.sum(func(a + kodd*h)))
        I2 = h * (S + 2*T)
        epsilon = np.abs((1/15) * (I2 - I1))
        I1 = I2
    
        tabela = "\n"
        tabela += f"N = {N} \n"
        tabela += f"I{i} = {I1}\n"
    
        tabela += f"Erro: {epsilon}"

        print(tabela)

In [ ]:
solveRomberg(f, 0, 1, 1e-6)

## Quadratura de Gauss

In [ ]:
def solveGaussLegendre(f, a, b, N):
    
    X, W = np.polynomial.legendre.leggauss(N)
    
    x = 0.5 * (b-a) * X + 0.5 * (b+a)
    w = 0.5 * (b-a) * W
    
    return f(x) @ w
    

In [ ]:
print(solveGaussLegendre(f, 0, 1, 10))

## Derivadas
$$ \frac{df}{dx}\approx \frac{f(x+\frac{h}{2})-f(x-\frac{h}{2})}{h}$$

In [ ]:
def derivada(f, a, b, N):
    x = np.linspace(a, b, N)
    h = x[1]-x[0]
    return (f(x + h/2) - f(x - h/2)) / h

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.linspace(-2*np.pi, 2*np.pi, 1000), derivada(f, -2*np.pi, 2*np.pi, 1000))
plt.plot(np.linspace(-2*np.pi, 2*np.pi, 1000), f(np.linspace(-2*np.pi, 2*np.pi, 1000)))